reference from: https://github.com/susanli2016/NLP-with-Python/blob/master/LDA_news_headlines.ipynb

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')

path = "/content/drive/My Drive/Colab Notebooks"

In [ ]:
pip install pandas

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/la_aptFinder_basic_info.csv', error_bad_lines=False,encoding='utf-8')
data_text = data[['description']]
data_text['index'] = data_text.index
data_text.rename(columns={'description':'headline_text'}, inplace = True)
data_text = data_text.dropna(subset=['headline_text'])
documents = data_text

In [ ]:
result_text = data[['description','aptFinder_url']]

In [ ]:
result_text = result_text.dropna(subset=['description'])

In [ ]:
length = len(documents)

In [ ]:
pip install gensim
pip install nltk
pip install numpy

In [ ]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2020)
import nltk
nltk.download('wordnet')

In [ ]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [ ]:
processed_docs = documents['headline_text'].map(preprocess)

Bag of words on dataset

In [ ]:
dictionary = gensim.corpora.Dictionary(processed_docs)

In [ ]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [ ]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

TF-IDF

In [ ]:
from gensim import corpora, models

tfidf = models.TfidfModel(bow_corpus)

In [ ]:
corpus_tfidf = tfidf[bow_corpus]

Run LDA using bag of words

In [ ]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=20, id2word=dictionary, passes=2, workers=2)
# for idx, topic in lda_model.print_topics(-1):
#     print('Topic: {} \nWords: {}'.format(idx, topic))

Runing LDA using TF-IDF

In [ ]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=20, id2word=dictionary, passes=2, workers=4)
# for idx, topic in lda_model_tfidf.print_topics(-1):
#     print('Topic: {} Word: {}'.format(idx, topic))

classification of topics

--evaluate bag of words

In [ ]:
result_topic = []
for i in range(0,length):
  result_topic.append(lda_model.get_document_topics(bow_corpus[i])[0])

In [ ]:
aptFinder_list = result_text['aptFinder_url'].values.tolist()

In [ ]:
k = [x[0] for x in result_topic]
sum_k = [x[1] for x in result_topic]
dic_k = {}
for i in k:
  if i in dic_k.keys():
    dic_k[i]+= 1
  else:
    dic_k[i] = 1
dic_aptFinder = {}
for ii in range(0,length):
  if k[ii] not in dic_aptFinder.keys():
    dic_aptFinder[k[ii]] = []
    dic_aptFinder[k[ii]].append(aptFinder_list[ii])
  elif k[ii] in dic_aptFinder.keys():
    dic_aptFinder[k[ii]].append(aptFinder_list[ii])

['https://www.apartmentfinder.com/California/Los-Angeles-Apartments/Hwh-Luxury-Living-Up-To-8-Weeks-Free-Apartments-yq5vkbv', 'https://www.apartmentfinder.com/California/Los-Angeles-Apartments/The-Adler-Apartments-mjj639h', 'https://www.apartmentfinder.com/California/Marina-Del-Rey-Apartments/Shores-Now-Touring-W-Amazing-Specials-Apartments-nllm53p', 'https://www.apartmentfinder.com/California/Los-Angeles-Apartments/The-Culver-Apartments-0rggg10', 'https://www.apartmentfinder.com/California/Los-Angeles-Apartments/Park-La-Brea-Apartments', 'https://www.apartmentfinder.com/California/Los-Angeles-Apartments/The-Emerson-Apartments', 'https://www.apartmentfinder.com/California/Los-Angeles-Apartments/Kodo-Apartments-0xlegy3', 'https://www.apartmentfinder.com/California/Los-Angeles-Apartments/Blackstone-Apartments-4-8-Weeks-Free-Apartments-45dexsl', 'https://www.apartmentfinder.com/California/Los-Angeles-Apartments/The-Pines-Apartments-vf2n07e', 'https://www.apartmentfinder.com/California/Los

In [ ]:
sum(sum_k)/length

In [ ]:
dic_k

In [ ]:
# for index, score in sorted(lda_model[bow_corpus[120]], key=lambda tup: -1*tup[1]):
#     print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 40)))

--evaluate TF-IDF

In [ ]:
# for index, score in sorted(lda_model_tfidf[bow_corpus[120]], key=lambda tup: -1*tup[1]):
#     print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 40)))

In [ ]:
result_topic2 = []
for i in range(0,length):
  result_topic2.append(lda_model_tfidf.get_document_topics(bow_corpus[i])[0])

In [ ]:
k2 = [x[0] for x in result_topic2]
sum_k2 = [x[1] for x in result_topic2]
dic_k2 = {}
for i in k2:
  if i in dic_k2.keys():
    dic_k2[i]+= 1
  else:
    dic_k2[i] = 1
dic_aptFinder2 = {}
for ii in range(0,length):
  if k2[ii] not in dic_aptFinder2.keys():
    dic_aptFinder2[k2[ii]] = []
    dic_aptFinder2[k2[ii]].append(aptFinder_list[ii])
  elif k2[ii] in dic_aptFinder2.keys():
    dic_aptFinder2[k2[ii]].append(aptFinder_list[ii])
# print(dic_aptFinder2)
# print(len(dic_aptFinder2[0]))

In [ ]:
dic_k2

In [ ]:
sum(sum_k2)/length

Test unseen document

In [ ]:
unseen_document = 'An apartment at holywood with big pool and easy to go to the airport.'
bow_vector1 = dictionary.doc2bow(preprocess(unseen_document))
bow_vector2 = dictionary.doc2bow(preprocess(unseen_document))
topic_test1 = lda_model.get_document_topics(bow_vector1)[0]
topic_test2 = lda_model_tfidf.get_document_topics(bow_vector2)[0]
topic_of_query = 0
if topic_test1[0] == topic_test2[0]:
  topic_of_query = topic_test1[0]
  if sum(sum_k)/length>sum(sum_k2)/length:
    res = dic_aptFinder[topic_of_query]
  else:
    res = dic_aptFinder2[topic_of_query]
else:
  if topic_test1[1]>topic_test2[1]:
    topic_of_query = topic_test1[0]
    res = dic_aptFinder[topic_of_query]
  else:
    topic_of_query = topic_test2[0]
    res = dic_aptFinder2[topic_of_query]

In [ ]:
res